### Naive Bayes Regression

In [29]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('722NaiveBayes').getOrCreate()

# import other packages
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.types import (StructField,StringType,IntegerType,StructType)

### Load Data

In [30]:
# integrate the 2 sources
# code adapted from https://www.geeksforgeeks.org/merge-two-dataframes-in-pyspark/
import functools 
def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)

In [31]:
# create a variable with the correct structure
data_schema = [StructField('Total Household Income',IntegerType(),True), StructField('Region',StringType(),True),
              StructField('Total Food Expenditure',IntegerType(),True), StructField('Main Source of Income',StringType(),True),
              StructField('Agricultural Household indicator',IntegerType(),True), StructField('Bread and Cereals Expenditure',IntegerType(),True),
              StructField('Total Rice Expenditure',IntegerType(),True), StructField('Meat Expenditure',IntegerType(),True),
              StructField('Total Fish and  marine products Expenditure',IntegerType(),True), StructField('Fruit Expenditure',IntegerType(),True),
              StructField('Vegetables Expenditure',IntegerType(),True), StructField('Restaurant and hotels Expenditure',IntegerType(),True),
              StructField('Alcoholic Beverages Expenditure',IntegerType(),True), StructField('Tobacco Expenditure',IntegerType(),True),
              StructField('Clothing, Footwear and Other Wear Expenditure',IntegerType(),True), StructField('Housing and water Expenditure',IntegerType(),True),
              StructField('Imputed House Rental Value',IntegerType(),True), StructField('Medical Care Expenditure',IntegerType(),True),
              StructField('Transportation Expenditure',IntegerType(),True), StructField('Communication Expenditure',IntegerType(),True),
              StructField('Education Expenditure',IntegerType(),True), StructField('Miscellaneous Goods and Services Expenditure',IntegerType(),True),
              StructField('Special Occasions Expenditure',IntegerType(),True), StructField('Crop Farming and Gardening expenses',IntegerType(),True),
              StructField('Total Income from Entrepreneurial Acitivites',IntegerType(),True), StructField('Household Head Sex',StringType(),True),
              StructField('Household Head Age',IntegerType(),True), StructField('Household Head Marital Status',StringType(),True),
              StructField('Household Head Highest Grade Completed',StringType(),True), StructField('Household Head Job or Business Indicator',StringType(),True),
              StructField('Household Head Occupation',StringType(),True), StructField('Household Head Class of Worker',StringType(),True),
              StructField('Type of Household',StringType(),True), StructField('Total Number of Family members',IntegerType(),True),
              StructField('Members with age less than 5 year old',IntegerType(),True), StructField('Members with age 5 - 17 years old',IntegerType(),True),
              StructField('Total number of family members employed',IntegerType(),True), StructField('Type of Building/House',StringType(),True),
              StructField('Type of Roof',StringType(),True), StructField('Type of Walls',StringType(),True),
              StructField('House Floor Area',IntegerType(),True), StructField('House Age',IntegerType(),True),
              StructField('Number of bedrooms',IntegerType(),True), StructField('Tenure Status',StringType(),True),
              StructField('Toilet Facilities',StringType(),True), StructField('Electricity',IntegerType(),True),
              StructField('Main Source of Water Supply',StringType(),True), StructField('Number of Television',IntegerType(),True),
              StructField('Number of CD/VCD/DVD',IntegerType(),True), StructField('Number of Component/Stereo set',IntegerType(),True),
              StructField('Number of Refrigerator/Freezer',IntegerType(),True), StructField('Number of Washing Machine',IntegerType(),True),
              StructField('Number of Airconditioner',IntegerType(),True), StructField('Number of Car, Jeep, Van',IntegerType(),True),
              StructField('Number of Landline/wireless telephones',IntegerType(),True), StructField('Number of Cellular phone',IntegerType(),True),
              StructField('Number of Personal Computer',IntegerType(),True), StructField('Number of Stove with Oven/Gas Range',IntegerType(),True),
              StructField('Number of Motorized Banca',IntegerType(),True), StructField('Number of Motorcycle/Tricycle',IntegerType(),True)
              ]

final_struct = StructType(fields=data_schema)

In [32]:
# And now we can read in the data using that schema. The other fields are read as integer. 
df1 = spark.read.option("header", True).csv('dataset/data_ncr.csv', schema=final_struct)
df2 = spark.read.option("header", True).csv('dataset/data_therest.csv', schema=final_struct)
statsdata = unionAll([df1, df2])

print("Combined dataset with updated dtypes:", statsdata.count(), "x", len(statsdata.columns))
statsdata.printSchema()

Combined dataset with updated dtypes: 41544 x 60
root
 |-- Total Household Income: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Total Food Expenditure: integer (nullable = true)
 |-- Main Source of Income: string (nullable = true)
 |-- Agricultural Household indicator: integer (nullable = true)
 |-- Bread and Cereals Expenditure: integer (nullable = true)
 |-- Total Rice Expenditure: integer (nullable = true)
 |-- Meat Expenditure: integer (nullable = true)
 |-- Total Fish and  marine products Expenditure: integer (nullable = true)
 |-- Fruit Expenditure: integer (nullable = true)
 |-- Vegetables Expenditure: integer (nullable = true)
 |-- Restaurant and hotels Expenditure: integer (nullable = true)
 |-- Alcoholic Beverages Expenditure: integer (nullable = true)
 |-- Tobacco Expenditure: integer (nullable = true)
 |-- Clothing, Footwear and Other Wear Expenditure: integer (nullable = true)
 |-- Housing and water Expenditure: integer (nullable = true)
 |-- Impute

### Clean Data

In [33]:
# Drop columns
Columns_To_Remove = ['Region', 'Number of Television', 'Number of CD/VCD/DVD', 'Number of Component/Stereo set', 
                     'Number of Refrigerator/Freezer', 'Number of Washing Machine', 'Number of Airconditioner', 
                     'Number of Car, Jeep, Van', 'Number of Landline/wireless telephones', 'Number of Cellular phone', 
                     'Number of Personal Computer', 'Number of Stove with Oven/Gas Range', 'Number of Motorized Banca', 
                     'Number of Motorcycle/Tricycle']
statsupdate = statsdata.drop(*Columns_To_Remove)

# Add New Column with Total Expenses
expenses_cols = ['Total Food Expenditure', 'Bread and Cereals Expenditure', 'Total Rice Expenditure', 'Meat Expenditure', 'Total Fish and  marine products Expenditure', 'Fruit Expenditure', 'Vegetables Expenditure', 'Restaurant and hotels Expenditure', 'Alcoholic Beverages Expenditure', 'Tobacco Expenditure', 'Clothing, Footwear and Other Wear Expenditure', 'Housing and water Expenditure', 'Medical Care Expenditure', 'Transportation Expenditure', 'Communication Expenditure', 'Education Expenditure', 'Miscellaneous Goods and Services Expenditure', 'Special Occasions Expenditure', 'Crop Farming and Gardening expenses']
statsupdate = statsupdate.withColumn('Total Expenditures', sum(statsupdate[col] for col in expenses_cols))

# Drop the other expenses columns, except Medical Care Expenditure
expenses_cols.remove('Medical Care Expenditure')
statsupdate = statsupdate.drop(*expenses_cols)
print("Updated dimensions after removing",len(expenses_cols)+len(Columns_To_Remove),"columns:",statsupdate.count(), "x", len(statsupdate.columns))

# remove na/nulls
statsupdate = statsupdate.na.fill('Not Specified')

# remove outliers
# calculate upper and lower bounds for outliers, adapted from internet
def calculate_bounds(df):
    bounds = {
        c: dict(
            zip(["q1", "q3"], df.approxQuantile(c, [0.25, 0.75], 0))
        )
        for c,d in zip(df.columns, df.dtypes) if d[1] == "int"
    }
    for c in bounds:
        iqr = bounds[c]['q3'] - bounds[c]['q1']
        bounds[c]['min'] = bounds[c]['q1'] - (iqr * 1.5)
        bounds[c]['max'] = bounds[c]['q3'] + (iqr * 1.5)
    return bounds
bounds = calculate_bounds(statsupdate)

# remove outliers for "Total Household Income"
c = "Total Household Income"
newstats = statsupdate.filter(F.col(c).between(bounds[c]['min'], bounds[c]['max']))

# remove outliers for "Medical Care Expenditure"
c = "Medical Care Expenditure"
newstats = newstats.filter(F.col(c).between(bounds[c]['min'], bounds[c]['max']))

# rename misspelled column
newstats = newstats.withColumnRenamed("Total Income from Entrepreneurial Acitivites", "Total Income from Entrepreneurial Activities")

# categorize
#create a list of the columns that are string types
categoricalColumns = [item[0] for item in newstats.dtypes if item[1].startswith('string') ]

#define a list of stages in your pipeline. The string indexer will be one stage
stages = []
#iterate through all categorical values, create a string indexer, assign new column name with 'Index' at end
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + ' Index')
    stages += [stringIndexer]
pipeline = Pipeline(stages = stages)
newstats = pipeline.fit(newstats).transform(newstats)

Updated dimensions after removing 32 columns: 41544 x 29


In [34]:
newstats.printSchema()

root
 |-- Total Household Income: integer (nullable = true)
 |-- Main Source of Income: string (nullable = false)
 |-- Agricultural Household indicator: integer (nullable = true)
 |-- Imputed House Rental Value: integer (nullable = true)
 |-- Medical Care Expenditure: integer (nullable = true)
 |-- Total Income from Entrepreneurial Activities: integer (nullable = true)
 |-- Household Head Sex: string (nullable = false)
 |-- Household Head Age: integer (nullable = true)
 |-- Household Head Marital Status: string (nullable = false)
 |-- Household Head Highest Grade Completed: string (nullable = false)
 |-- Household Head Job or Business Indicator: string (nullable = false)
 |-- Household Head Occupation: string (nullable = false)
 |-- Household Head Class of Worker: string (nullable = false)
 |-- Type of Household: string (nullable = false)
 |-- Total Number of Family members: integer (nullable = true)
 |-- Members with age less than 5 year old: integer (nullable = true)
 |-- Members wit

### Naive Bayes

In [35]:
from pyspark.ml.classification import NaiveBayes
# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [36]:
assembler = VectorAssembler(
    inputCols=["Total Household Income", "Main Source of Income Index", 
            "Agricultural Household indicator","Imputed House Rental Value",
            "Total Income from Entrepreneurial Activities",
            "Household Head Age", "Total Number of Family members",
            "Members with age less than 5 year old","Members with age 5 - 17 years old",
            "Total number of family members employed", "House Floor Area",
             "House Age","Number of bedrooms",
             "Electricity","Total Expenditures",
             "Main Source of Income Index","Household Head Sex Index",
             "Household Head Marital Status Index","Household Head Highest Grade Completed Index",
             "Household Head Job or Business Indicator Index",
             "Household Head Class of Worker Index","Type of Household Index",
             "Type of Building/House Index","Type of Roof Index","Type of Walls Index",
             "Tenure Status Index","Toilet Facilities Index",
             "Main Source of Water Supply Index"],
    outputCol="features")
output = assembler.transform(newstats)
labelIndexer = StringIndexer(inputCol = "Medical Care Expenditure", outputCol = "label")
output = labelIndexer.fit(output).transform(output)

In [38]:
final_data = output.select("features",'label')
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [39]:
# Train a naive Bayes model.
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")
nb = nb.fit(train_data)


In [40]:
prediction = nb.transform(test_data)
prediction.show(5)


22/10/12 22:38:42 WARN DAGScheduler: Broadcasting large task binary with size 1790.8 KiB


+--------------------+------+--------------------+--------------------+----------+
|            features| label|       rawPrediction|         probability|prediction|
+--------------------+------+--------------------+--------------------+----------+
|(28,[0,1,2,3,4,5,...|1314.0|[-916035.58472603...|[0.0,0.0,0.0,0.0,...|    3611.0|
|(28,[0,1,2,3,4,5,...|2067.0|[-1307555.2774634...|[0.0,0.0,0.0,0.0,...|    3669.0|
|(28,[0,1,2,3,4,5,...|3389.0|[-2435699.9117475...|[0.0,0.0,0.0,0.0,...|    3441.0|
|(28,[0,1,2,3,4,5,...|5473.0|[-807817.66904763...|[0.0,0.0,0.0,0.0,...|    4038.0|
|(28,[0,1,2,3,4,5,...|1901.0|[-1029926.8424436...|[0.0,0.0,0.0,0.0,...|    2598.0|
+--------------------+------+--------------------+--------------------+----------+
only showing top 5 rows



In [50]:
# adapted from https://www.datatechnotes.com/2021/12/mllib-naive-bayes-classification.html
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator=MulticlassClassificationEvaluator(predictionCol="prediction")
accuracy = evaluator.evaluate(prediction)
print("Prediction Accuracy: {:.8f}".format(accuracy))


22/10/12 22:49:18 WARN DAGScheduler: Broadcasting large task binary with size 1792.2 KiB


Prediction Accuracy: 0.00002418


In [55]:
# Run several more iterations and reducing 
def reRunNaiveBayes (output):
    # re-split data
    final_data = output.select("features",'label')
    train_data,test_data = final_data.randomSplit([0.7,0.3])
    # re-run model
    nb = NaiveBayes(smoothing=0.5, modelType="multinomial")
    nb = nb.fit(train_data)
    # predict
    prediction = nb.transform(test_data)
    # report accuracy
    evaluator=MulticlassClassificationEvaluator(predictionCol="prediction")
    accuracy = evaluator.evaluate(prediction)
    print("Prediction Accuracy: {:.8f}".format(accuracy))

In [56]:
reRunNaiveBayes(output)

22/10/12 22:54:30 WARN DAGScheduler: Broadcasting large task binary with size 1794.8 KiB


Prediction Accuracy: 0.00009635


In [57]:
# decrease features
assembler = VectorAssembler(
    inputCols=["Total Household Income", 
               "Total Expenditures",
               "Main Source of Income Index", 
                "Household Head Age", 
                "Household Head Sex Index",
                "Household Head Marital Status Index",
                "Main Source of Water Supply Index"],
    outputCol="features")
output = assembler.transform(newstats)
labelIndexer = StringIndexer(inputCol = "Medical Care Expenditure", outputCol = "label")
output = labelIndexer.fit(output).transform(output)

In [62]:
output.head(1)

[Row(Total Household Income=301900, Main Source of Income='Wage/Salaries', Agricultural Household indicator=0, Imputed House Rental Value=72000, Medical Care Expenditure=480, Total Income from Entrepreneurial Activities=0, Household Head Sex='Female', Household Head Age=73, Household Head Marital Status='Widowed', Household Head Highest Grade Completed='Elementary Graduate', Household Head Job or Business Indicator='No Job/Business', Household Head Occupation='Not Specified', Household Head Class of Worker='Not Specified', Type of Household='Extended Family', Total Number of Family members=4, Members with age less than 5 year old=0, Members with age 5 - 17 years old=0, Total number of family members employed=1, Type of Building/House='Single house', Type of Roof='Strong material(galvanized,iron,al,tile,concrete,brick,stone,asbestos)', Type of Walls='Strong', House Floor Area=25, House Age=15, Number of bedrooms=2, Tenure Status='Own or owner-like possession of house and lot', Toilet Fa

In [63]:
reRunNaiveBayes(output)

Prediction Accuracy: 0.00076062


In [64]:
reRunNaiveBayes(output)

Prediction Accuracy: 0.00078560


In [65]:
# decrease features even more
assembler = VectorAssembler(
    inputCols=["Total Household Income", 
                "Household Head Sex Index",
                "Household Head Marital Status Index"],
    outputCol="features")
output = assembler.transform(newstats)
labelIndexer = StringIndexer(inputCol = "Medical Care Expenditure", outputCol = "label")
output = labelIndexer.fit(output).transform(output)

In [66]:
reRunNaiveBayes(output)
reRunNaiveBayes(output)

Prediction Accuracy: 0.00315711


Prediction Accuracy: 0.00318033


In [68]:
# Stop session 
spark.stop()